In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
from scipy.io import wavfile
import warnings
from pydub import AudioSegment
import ffmpeg

In [ ]:
path1='C:/Users/todtj/OneDrive/바탕 화면/TED1/'   # 원본 음성파일
path2='C:/Users/todtj/OneDrive/바탕 화면/script1/' # 크롤링해서 저장된 폴더
path3='C:/Users/todtj/OneDrive/바탕 화면/save/'   # 저장될공간

## 문장단위로 만드는 코드


In [ ]:
path1='C:/Users/todtj/OneDrive/바탕 화면/TED1/'   # 원본 음성파일
path2='C:/Users/todtj/OneDrive/바탕 화면/script1/' # 크롤링해서 저장된 폴더
path3='C:/Users/todtj/OneDrive/바탕 화면/save/'   # 저장될공간
file_list1=os.listdir(path1)
file_list2=os.listdir(path2)
for g in range(len(file_list2)):
    voice1=path1+file_list1[g]
    sound = AudioSegment.from_file(voice1,frame_rate=16000)
    script1=pd.read_csv(path2+file_list2[g])
    if script1.columns[0]=='Unnamed: 0':
        del script1['Unnamed: 0']
    if len(script1.columns)==3:
        script1['sentence2']=''
    elif len(script1.columns)==4:
        script1['sentence2'].fillna('',inplace=True)
    script1['sentence3']=script1['sentence1']+' '+script1['sentence2']
## start##
    spl=script1['start'].str.split(':')
    empty=[]
    for i in range(len(script1)):
        minute=float(spl[i][1])*60
        seconds=float(spl[i][2])
        m_s=minute+seconds
        empty.append(m_s)
    script1['start_seconds']=empty
## end##
    spl=script1['end'].str.split(':')
    empty=[]
    for i in range(len(script1)):
        minute=float(spl[i][1])*60
        seconds=float(spl[i][2])
        m_s=minute+seconds
        empty.append(m_s)
    script1['end_seconds']=empty
    script1['start_seconds1']=script1['start_seconds']
    script1['end_seconds1']=script1['end_seconds']
    script1['duration']=script1['end_seconds1']-script1['start_seconds1']
    sentence_cut=[]
    start_cut=[]
    end_cut=[]
    a=''
    b=0
    c=0
    count=0
    
    for i in range(len(script1)):
        if '.' in script1['sentence3'][i]:
            a=a+ script1['sentence3'][i]
            b=script1['start_seconds1'][i-count]
            c=script1['end_seconds1'][i]
            sentence_cut.append(a)
            start_cut.append(b)
            end_cut.append(c)
            a=''
            c=0 
            count=0 
        else:
            a=a+ ' ' + script1['sentence3'][i]
            count=count+1        
    final=pd.DataFrame([sentence_cut,start_cut,end_cut]).T
    final.columns=['sentence','start','end']
    final['duration']=final['end']-final['start']
    for i in range(len(final)):
        sound_sliced=sound[(final['start'][i]*1000-100):final['end'][i]*1000]
        sound_sliced.export(path3+file_list1[g].replace(".wav","")+"_"+str(i)+".wav", format="wav",bitrate=16)
        #y,sr= librosa.load(voice1,offset=final['start'][i],duration=final['duration'][i],sr=16000)
        #librosa.output.write_wav(path3+file_list1[g].replace(".wav","")+"_"+str(i)+".wav",y,sr)
    final.to_csv(path3+file_list1[g]+'.csv',index=False)


# chunk 단위로 duration까지만들기

# 주어진 형식대로 자르기

In [ ]:
path1='C:/Users/todtj/OneDrive/바탕 화면/TED1/'   # 원본 음성파일
path2='C:/Users/todtj/OneDrive/바탕 화면/script1/' # 크롤링해서 저장된 폴더
path3='C:/Users/todtj/OneDrive/바탕 화면/save/'   # 저장될공간
file_list1=os.listdir(path1)
file_list2=os.listdir(path2)
for g in range(len(file_list2)):
    #voice1=path1+file_list1[g]
    #sound = AudioSegment.from_file(voice1,frame_rate=16000)
    script1=pd.read_csv(path2+file_list2[g],encoding='utf-8')
    if script1.columns[0]=='Unnamed: 0':
        del script1['Unnamed: 0']
    if len(script1.columns)==3:
        script1['sentence2']=''
    elif len(script1.columns)==4:
        script1['sentence2'].fillna('',inplace=True)
    script1['sentence3']=script1['sentence1']+' '+script1['sentence2']
## start##
    spl=script1['start'].str.split(':')
    empty=[]
    for i in range(len(script1)):
        minute=float(spl[i][1])*60
        seconds=float(spl[i][2])
        m_s=minute+seconds
        empty.append(m_s)
    script1['start_seconds']=empty
## end##
    spl=script1['end'].str.split(':')
    empty=[]
    for i in range(len(script1)):
        minute=float(spl[i][1])*60
        seconds=float(spl[i][2])
        m_s=minute+seconds
        empty.append(m_s)
    script1['end_seconds']=empty
    script1['start_seconds1']=script1['start_seconds']+12.9
    script1['end_seconds1']=script1['end_seconds']+12.9
    final=pd.DataFrame([script1.sentence3,script1.start_seconds1,script1.end_seconds1]).T
    final.columns=['sentence','start','end']
    final['duration']=final['end']-final['start']
    for i in range(len(final)):
        sound_sliced=sound[(final['start'][i]*1000-400):final['end'][i]*1000]
        sound_sliced.export(path3+file_list1[g].replace(".wav","")+"_"+str(i)+".wav", format="wav",bitrate=16)
        y,sr= librosa.load(voice1,offset=final['start'][i],duration=final['duration'][i],sr=16000)
        librosa.output.write_wav(path3+file_list1[g].replace(".wav","")+"_"+str(i)+".wav",y,sr)
    final['talks_id']= file_list1[g].replace('.wav','')
    final.to_csv(path3+file_list1[g]+'.csv',index=False)
    break

#  모든 csv파일들 합치기

In [ ]:
for i in range(len(file_list2)):
    a=path2+file_list2[i]
    df=pd.read_csv(a)
    df['talk_id']